In [1]:
import json
import pandas as pd
from grant_parser import clean_workbook

In [2]:
df = clean_workbook('data/Optimism GovFund Grants_ Public Delivery Tracking.xlsx')

In [3]:
df.groupby('Grants Season or Mission')['OP Total Amount'].sum() / 1_000_000

Grants Season or Mission
Grants Season 1      42.630770
Grants Season 2      13.118661
Grants Season 3       4.529500
Grants Season 4       6.591123
Grants Season 5       6.880874
Grants Season 6      16.109290
Grants Season 7       9.839884
Missions Season 4     2.487276
Name: OP Total Amount, dtype: float64

In [5]:
with open('scraper/data/urls.json', 'r') as f:
    urls = json.load(f)
scraped_urls = urls['scraped']

In [6]:
df_links = df[df['OP Total Amount'] > 0].copy()
df_links['html'] = df_links['Link'].apply(lambda x: scraped_urls.get(x, None))
df_links['html'].value_counts()

html
experimenting-with-futarchy-for-optimism-grant-allocation-decisions.html             5
charmverse-proposal-ether-fi-op-mainnet-lrt-grant-46198635300198077.html             2
charmverse-proposal-ponder-surveys-for-superchain-builders-43349586555154374.html    2
charmverse-page-15106949611493015.html                                               2
charmverse-proposal-optimism-govquests-8512541302772494.html                         2
                                                                                    ..
charmverse-proposal-page-6225720466674716.html                                       1
charmverse-proposal-page-27013511701775395.html                                      1
charmverse-proposal-bcaedc5b-fbc5-46af-a9e4-d292bbf1cde2.html                        1
charmverse-proposal-page-8035332025592417.html                                       1
charmverse-proposal-chainsecurity-velodrome-superswaps-06394677827148398.html        1
Name: count, Length: 497, dtype: int64

In [7]:
# df_links[['Project Name', 'html']].drop_duplicates().to_csv('scraper/project_html.csv', index=False)

In [9]:
df_matches = pd.read_csv('scraper/data/project_matches.csv')
matched_projects = df_matches.set_index('html')['oso_slug'].to_dict()
df_links['oso_slug'] = df_links['html'].apply(lambda x: matched_projects.get(x, None))
df_links

,Project Name,Grants Season or Mission,Status,Initial delivery date,OP Delivered,OP Total Amount,Intent,Link,Other metadata,html,oso_slug
0,Uniswap V3,Grants Season 1,Sent,2022-09-07 00:00:00,NaN,1000000.0,<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Cycle #"": ""Cycle 1 (Phase 0)"", ""Incentive Pr...",gf-phase-0-proposal-uniswap.html,uniswap
1,Oneinch,Grants Season 1,Sent,2022-08-15 00:00:00,NaN,300000.0,<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Incentive Program Launched?"": ""Yes"", ""Propos...",gf-phase-0-proposal-1inch-network.html,1-inch
2,Celer,Grants Season 1,Sent,2022-08-04 00:00:00,NaN,1000000.0,<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Proposal Link"": ""https://gov.optimism.io/t/g...",gf-phase-0-proposal-celer-network.html,celer
3,Aave,Grants Season 1,Sent,2022-08-03 00:00:00,NaN,300000.0,<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Incentive Program Launched?"": ""Yes"", ""Propos...",gf-phase-0-proposal-aave.html,aave
4,Chainlink,Grants Season 1,Sent,2022-08-03 00:00:00,NaN,1000000.0,<NA>,https://gov.optimism.io/t/gf-phase-0-proposal-...,"{""Proposal Link"": ""https://gov.optimism.io/t/g...",gf-phase-0-proposal-chainlink.html,chainlink
...,...,...,...,...,...,...,...,...,...,...,...
1638,Kyo Finance Audit Grant Application,Grants Season 7,NaN,NaN,NaN,92806.0,Audits,https://app.charmverse.io/op-grants/kyo-financ...,"{""Incentive Program Launched?"": ""Builders"", ""L...",charmverse-proposal-kyo-finance-audit-grant-ap...,unknown
1639,Kyo Finance Audit Grant Application,Grants Season 7,NaN,NaN,NaN,23202.0,Audits,https://app.charmverse.io/op-grants/kyo-financ...,"{""Incentive Program Launched?"": ""Growth"", ""L2 ...",charmverse-proposal-kyo-finance-audit-grant-ap...,unknown
1640,Vfat subsidised audit 2nd batch,Grants Season 7,NaN,NaN,NaN,87000.0,Audits,https://app.charmverse.io/op-grants/vfat-subsi...,"{""Incentive Program Launched?"": ""Growth"", ""L2 ...",charmverse-proposal-vfat-subsidised-audit-2nd-...,unknown
1641,ChainSecurity: grant request for Neulock audit,Grants Season 7,NaN,NaN,NaN,45000.0,Audits,https://app.charmverse.io/op-grants/chainsecur...,"{""Incentive Program Launched?"": ""Growth"", ""L2 ...",charmverse-proposal-chainsecurity-grant-reques...,unknown


In [10]:
df_links.to_csv("matched_tokenhouse_projects.csv")

In [16]:
df_not_approved = df[(df['Link'].notnull()) & (df['OP Total Amount'].isnull())]
df_not_approved

,Project Name,Grants Season or Mission,Status,Initial delivery date,OP Delivered,OP Total Amount,Intent,Link,Other metadata
25,Optimistic Railway,Grants Season 1,Not-passed,---,NaN,NaN,<NA>,https://gov.optimism.io/t/ready-gf-phase-1-pro...,"{""Cycle #"": ""Cycle 2 (Phase 1)"", ""Incentive Pr..."
27,GYSR,Grants Season 1,Not-passed,---,NaN,NaN,<NA>,https://gov.optimism.io/t/ready-gf-phase-1-pro...,"{""Incentive Program Launched?"": ""---"", ""Propos..."
29,Raptor,Grants Season 1,Not-passed,---,NaN,NaN,<NA>,https://gov.optimism.io/t/ready-gf-phase-1-pro...,"{""Incentive Program Launched?"": ""---"", ""Propos..."
31,Summa,Grants Season 1,Not-passed,---,NaN,NaN,<NA>,https://gov.optimism.io/t/ready-gf-phase-1-pro...,"{""Proposal Link"": ""https://gov.optimism.io/t/r..."
34,Ooki Protocol,Grants Season 1,Not-passed,---,NaN,NaN,<NA>,https://gov.optimism.io/t/ready-gf-phase-1-pro...,"{""Incentive Program Launched?"": ""---"", ""Propos..."
...,...,...,...,...,...,...,...,...,...
1754,Haifu Audit | Hacken,Grants Season 7,Not-passed,NaN,NaN,NaN,Audits,https://app.charmverse.io/op-grants/haifu-audi...,{}
1755,Eigenpie Audit | Hacken,Grants Season 7,Not-passed,NaN,NaN,NaN,Audits,https://app.charmverse.io/op-grants/eigenpie-a...,{}
1756,Droplinked Audit | Hacken,Grants Season 7,Not-passed,NaN,NaN,NaN,Audits,https://app.charmverse.io/op-grants/droplinked...,{}
1757,Seamless Protocol Audit Application,Grants Season 7,Not-passed,NaN,NaN,NaN,Audits,https://app.charmverse.io/op-grants/seamless-p...,{}


In [17]:
df_not_approved.to_csv("unapproved_tokenhouse_projects.csv")